In [10]:
from google.colab import files

# Upload files interactively
uploaded = files.upload()




Saving Customers.csv to Customers (5).csv
Saving Products.csv to Products (5).csv
Saving Transactions.csv to Transactions (5).csv


In [ ]:
pip install numpy

In [13]:
# List uploaded files
print(uploaded.keys())

import pandas as pd
import os
import logging
logging.basicConfig(level=logging.INFO)
logging.info("Processing customer X...")
print(os.listdir())


# Load the uploaded files
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

# Verify the data
print("Customers:\n", customers.head())
print("Products:\n", products.head())
print("Transactions:\n", transactions.head())






dict_keys(['Customers (5).csv', 'Products (5).csv', 'Transactions (5).csv'])
['.config', 'Products (3).csv', 'Products (2).csv', 'Transactions (3).csv', 'Products (4).csv', 'Transactions (1).csv', 'Products (5).csv', 'Customers (4).csv', 'Transactions (4).csv', 'Customers (3).csv', 'Transactions.csv', 'Customers.csv', 'Customers (1).csv', 'Transactions (5).csv', 'Products.csv', 'Customers (2).csv', 'Products (1).csv', 'Transactions (2).csv', 'Customers (5).csv', 'sample_data']
Customers:
   CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
Products:
   ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    Acti

In [14]:


import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np


# Confirm the merge worked
print("Merged Transactions:\n", transactions.head())

# Feature Engineering: Create a customer-product matrix
customer_product_matrix = transactions.pivot_table(
    index='CustomerID',
    columns='ProductID',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)

# Normalize the customer-product matrix
scaler = StandardScaler()
normalized_matrix = scaler.fit_transform(customer_product_matrix)

# Compute cosine similarity between customers
similarity_matrix = cosine_similarity(normalized_matrix)

# Convert similarity matrix to a DataFrame for easy handling
similarity_df = pd.DataFrame(
    similarity_matrix,
    index=customer_product_matrix.index,
    columns=customer_product_matrix.index
)

# Function to get top 3 similar customers for a given customer
def get_top_3_similar_customers(customer_id):
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:4]
    return list(zip(similar_customers.index, similar_customers.values))

# Generate Lookalike.csv for the first 20 customers (C0001 - C0020)
lookalike_data = {}
for customer_id in customers['CustomerID'][:20]:
    lookalike_data[customer_id] = get_top_3_similar_customers(customer_id)

# Create a DataFrame to store the results
lookalike_df = pd.DataFrame({
    'CustomerID': lookalike_data.keys(),
    'SimilarCustomers': [x for x in lookalike_data.values()]
})

# Save the lookalike results to a CSV file
lookalike_df.to_csv("Lookalike.csv", index=False)

print("Lookalike.csv has been generated!")


Merged Transactions:
   TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue   Price  
0      300.68  300.68  
1      300.68  300.68  
2      300.68  300.68  
3      601.36  300.68  
4      902.04  300.68  
Lookalike.csv has been generated!
